In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# [TODO] Add your H1 title heading here

{TODO: Update the links below.} 

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/notebook_template.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/notebook_template.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/notebook_template.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>                                                                                               
</table>

**_NOTE_**: This notebook has been tested in the following environment:

* Python version = 3.9

## Overview

{TODO: Include a paragraph or two explaining what this example demonstrates, who should be interested in it, and what you need to know before you get started.}

Learn more about [web-doc-title](linkback-to-webdoc-page). {TODO: if more than one primary feature, add tag/linkback for each one}

### Objective

In this tutorial, you learn how to set up the repo, launch your first training and predicition pipeline, and analyse the results:

This tutorial uses the following Google Cloud ML services and resources:

- *{TODO: Add high level bullets for the services/resources demonstrated; e.g., Vertex AI Training}*


The steps performed include:

- *{TODO: Add high level bullets for the steps of performed in the notebook}*

### Dataset

{TODO: Include a paragraph with Dataset information and where to obtain it.} 

{TODO: Make sure the dataset is accessible to the public. **Googlers**: Add your dataset to the [public samples bucket](http://goto/cloudsamples#sample-storage-bucket) within gs://cloud-samples-data/vertex-ai, if it doesn't already exist there.}

### Costs 

{TODO: Update the list of billable products that your tutorial uses.}

This tutorial uses billable components of Google Cloud:

* Vertex AI
* {TODO: BigQuery}
* Cloud Storage

{TODO: Include links to pricing documentation for each product you listed above.
 NOTE: If you use BigQuery or Dataflow, you need to add this to the pricing.
}

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing),
{ TODO: [BigQuery pricing](https://cloud.google.com/bigquery/pricing), }
and [Cloud Storage pricing](https://cloud.google.com/storage/pricing), 
and use the [Pricing Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

### Prerequisites

- [Pyenv](https://github.com/pyenv/pyenv#installation) for managing Python versions
- [Google Cloud SDK (gcloud)](https://cloud.google.com/sdk/docs/quickstart)
- Make
- [Poetry](https://python-poetry.org)

## Clone Turbo Templates repository

In [1]:
# Clone a Git repository
!git clone -b develop https://github.com/teamdatatonic/vertex-pipelines-end-to-end-samples

Cloning into 'vertex-pipelines-end-to-end-samples'...
remote: Enumerating objects: 3866, done.
remote: Counting objects: 100% (2156/2156), done.
remote: Compressing objects: 100% (843/843), done.
remote: Total 3866 (delta 1311), reused 1807 (delta 1202), pack-reused 1710
Receiving objects: 100% (3866/3866), 3.05 MiB | 3.94 MiB/s, done.
Resolving deltas: 100% (2173/2173), done.


In [2]:
%cd vertex-pipelines-end-to-end-samples/

/Users/donatasmickevicius/code/vertex-pipelines-end-to-end-samples/vertex-pipelines-end-to-end-samples


## Installation

Install the following packages required to execute this notebook. 

{TODO: Suggest using the latest major GA version of each package; i.e., --upgrade}

In [3]:
# Install the correct Python version
! pyenv install -skip-existing

# configure poetry 
! poetry config virtualenvs.prefer-active-python true

#Install poetry dependencies for ML pipelines
! make install

# Install pre-commit hooks
! cd pipelines && poetry run pre-commit install


Usage: pyenv install [-f] [-kvp] <version>...
       pyenv install [-f] [-kvp] <definition-file>
       pyenv install -l|--list
       pyenv install --version

  -l/--list          List all available versions
  -f/--force         Install even if the version appears to be installed already
  -s/--skip-existing Skip if the version appears to be installed already

  python-build options:

  -k/--keep          Keep source tree in $PYENV_BUILD_ROOT after installation
                     (defaults to $PYENV_ROOT/sources)
  -p/--patch         Apply a patch from stdin before building
  -v/--verbose       Verbose mode: print compilation status to stdout
  --version          Show version of python-build
  -g/--debug         Build a debug version

For detailed information on installing Python versions with
python-build, including a list of environment variables for adjusting
compilation, see: https://github.com/pyenv/pyenv#readme

Creating virtualenv pipelines in /Users/donatasmickevicius/code/v

### Colab only: Uncomment the following cell to restart the kernel.

In [4]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## 1. Setup

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

2. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

3. If you are running this notebook locally, you need to install the [Cloud SDK](https://cloud.google.com/sdk).

#### Set your project ID

**If you don't know your project ID**, try the following:
* Run `gcloud config list`.
* Run `gcloud projects list`.
* See the support page: [Locate the project ID](https://support.google.com/googleapi/answer/7014113)

In [5]:
vertex_project_id = "dt-donatas-sandbox-dev"  # @param {type:"string"}

# Set the project id
! gcloud config set project {vertex_project_id}

# Set the environment variable
%env VERTEX_PROJECT_ID = {vertex_project_id}

Updated property [core/project].


Updates are available for some Google Cloud CLI components.  To install them,
please run:
  $ gcloud components update

env: VERTEX_PROJECT_ID=dt-donatas-sandbox-dev


#### Location

You can also change the `VERTEX_LOCATION` variable used by Vertex AI. Learn more about [Vertex AI locations](https://cloud.google.com/vertex-ai/docs/general/locations).

In [6]:
vertex_location = "europe-west2"  # @param {type: "string"}

# Set the environment variable
%env VERTEX_LOCATION={vertex_location}

env: VERTEX_LOCATION=europe-west2


#### Resource Suffix

Specify the `RESOURCE_SUFFIX` (e.g. `<your name>`) to facilitate running concurrent pipelines in the same Google Cloud project. 
This is important to change if you are working in a team to avoid overwriting resources during development 

In [7]:
resource_suffix="default" # @param {type: "string"}

# Set the environment variable
%env RESOUCE_SUFFIX=resource_suffix

env: RESOUCE_SUFFIX=resource_suffix


#### Vertex Pipelines Service Account

We need to set the `VERTEX_SA_EMAIL` that is in the Google Cloud project so that this service account can run the pipelines for us.

In [8]:
vertex_sa_email=f"vertex-pipelines@{vertex_project_id}.iam.gserviceaccount.com"

# Set the environment variable
%env VERTEX_SA_EMAIL={vertex_sa_email}

env: VERTEX_SA_EMAIL=vertex-pipelines@dt-donatas-sandbox-dev.iam.gserviceaccount.com


#### Vertex Pipelines Root Bucket

We need to set the `VERTEX_PIPELINE_ROOT` that is in the Google Cloud project which is used to stage pipeline artifacts.

In [9]:
vertex_pipeline_root=f"gs://${vertex_project_id}-pl-root"

# Set the environment variable
%env VERTEX_PIPELINE_ROOT={vertex_pipeline_root}

env: VERTEX_PIPELINE_ROOT=gs://$dt-donatas-sandbox-dev-pl-root


#### Container Image Registry

We need to set the `CONTAINER_IMAGE_REGISTRY` that is container image repository for training/serving container images.

In [10]:
container_image_registry=f"{vertex_location}-docker.pkg.dev/{vertex_project_id}/vertex-images"

# Set the environment variable
%env CONTAINER_IMAGE_REGISTRY={container_image_registry}

env: CONTAINER_IMAGE_REGISTRY=europe-west2-docker.pkg.dev/dt-donatas-sandbox-dev/vertex-images


### Authenticate your Google Cloud account

Depending on your Jupyter environment, you may have to manually authenticate. Follow the relevant instructions below.

**1. Vertex AI Workbench**
* Do nothing as you are already authenticated.

**2. Local JupyterLab instance, uncomment and run:**

In [11]:
# ! gcloud auth login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=k6jMTq96cILXIHTYaj26JHwqvHn6Ey&access_type=offline&code_challenge=whoq3ZN3MktqtUR1PwXG5jV0GljumM8qPT6GQpl-l5w&code_challenge_method=S256


You are now logged in as [donatas.mickevicius@datatonic.com].
Your current project is [dt-donatas-sandbox-dev].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


**3. Colab, uncomment and run:**

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

**4. Service account or other**
* See how to grant Cloud Storage permissions to your service account at https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples.

### Infrastructure deployment using terraform.

Install Terraform on your local machine. We recommend using [`tfswitch`](https://tfswitch.warrensbox.com/) to automatically choose and download an appropriate version for you (run `tfswitch` from the [`terraform/envs/dev`](terraform/envs/dev/) directory).

Now you can deploy the infrastructure using Terraform:

#### Create tfstate bucket

Before provisioning our infrastructure we need to create Google Cloud Storage (GCS) bucket that will be used to store the state files for Terraform deployments.

- *{Note to notebook author: For any user-provided strings that need to be unique (like bucket names or model ID's), append "-unique" to the end so proper testing can occur}*

In [12]:
tf_state_bucket_uri = f"gs://{vertex_project_id}-tfstate"  # @param {type:"string"}

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [13]:
! gsutil mb -l {vertex_location} -p {vertex_project_id} {tf_state_bucket_uri}

Creating gs://dt-donatas-sandbox-dev-tfstate/...
ServiceException: 409 A Cloud Storage bucket named 'dt-donatas-sandbox-dev-tfstate' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.


### Deploy required infrastructure

In [14]:
!echo ${VERTEX_PROJECT_ID}

dt-donatas-sandbox-dev


**Initialize Backend**

In [ ]:
! terraform -chdir=terraform/envs/dev init -backend-config="bucket=${VERTEX_PROJECT_ID}-tfstate" 

**Terraform Plan**

In [ ]:
! terraform -chdir=terraform/envs/dev plan -var "project_id=${VERTEX_PROJECT_ID}" -var "region=${VERTEX_LOCATION}"

**Terraform Apply** Please check tf plan output to see what infrastructure will be provisioned. If everything looks ok uncomment and run cell below.

In [ ]:
# ! terraform -chdir=terraform/envs/dev apply -var "project_id=${VERTEX_PROJECT_ID}" -var "region=${VERTEX_LOCATION}" -auto-approve -lock=false

## 2. Example ML Pipelines

This repository contains example ML training and prediction pipelines for two popular frameworks (XGBoost/sklearn and Tensorflow) using the popular [Chicago Taxi Dataset](https://console.cloud.google.com/marketplace/details/city-of-chicago-public-data/chicago-taxi-trips). The details of these can be found in the [separate README](pipelines/README.md).

#### Pre-requisites

Before you can run these example pipelines successfully there are a few additional things you will need to deploy (they have not been included in the Terraform code as they are specific to these pipelines)

1. Create a new BigQuery dataset for the Chicago Taxi data:

In [26]:
! bq --location=${VERTEX_LOCATION} mk --dataset "${VERTEX_PROJECT_ID}:chicago_taxi_trips"

BigQuery error in mk operation: Dataset 'dt-donatas-sandbox-
dev:chicago_taxi_trips' already exists.


2. Create a new BigQuery dataset for data processing during the pipelines:

In [28]:
! bq --location=${VERTEX_LOCATION} mk --dataset "${VERTEX_PROJECT_ID}:preprocessing"

BigQuery error in mk operation: Dataset 'dt-donatas-sandbox-dev:preprocessing'
already exists.


3. Set up a BigQuery transfer job to mirror the Chicago Taxi dataset to your project

In [30]:
! bq mk --transfer_config \
  --project_id=${VERTEX_PROJECT_ID} \
  --data_source="cross_region_copy" \
  --target_dataset="chicago_taxi_trips" \
  --display_name="Chicago taxi trip mirror" \
  --params='{"source_dataset_id":"'"chicago_taxi_trips"'","source_project_id":"'"bigquery-public-data"'"}' \
  --quiet=true


https://www.gstatic.com/bigquerydatatransfer/oauthz/auth?client_id=433065040935-uvs8ptbn0udhuartdhl4nn1l27t5v3qg.apps.googleusercontent.com&scope=https://www.googleapis.com/auth/bigquery&redirect_uri=urn:ietf:wg:oauth:2.0:oob&response_type=version_info
Please copy and paste the above URL into your web browser and follow the instructions to retrieve a version_info.
Enter your version_info here: ^C


In [ ]:
"source_project_id":"'"bigquery-public-data"'"

In [ ]:
! pip install google-cloud-bigquery-datatransfer

In [35]:
!echo $GOOGLE_APPLICATION_CREDENTIALS

In [36]:
! gcloud auth application-default login 

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=5HwStYisqE4FHgRbPiNDCIMrtmmJ4o&access_type=offline&code_challenge=4d8hOLhGRdD86uNzFAM_NpHkz6S70HIqB0E5XQhXpBs&code_challenge_method=S256


Credentials saved to file: [/Users/donatasmickevicius/.config/gcloud/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Quota project "dt-donatas-sandbox-dev" was added to ADC which can be used by Google client libraries for billing and quota. Note that some services may still bill the pr

In [46]:
from google.cloud import bigquery_datatransfer

transfer_client = bigquery_datatransfer.DataTransferServiceClient()

destination_project_id = vertex_project_id
destination_dataset_id = "chicago_taxi_trips"
source_project_id = "bigquery-public-data"
source_dataset_id = "chicago_taxi_trips"
transfer_config = bigquery_datatransfer.TransferConfig(
    destination_dataset_id=destination_dataset_id,
    display_name="Chicago taxi trip mirror",
    data_source_id="cross_region_copy",
    params={
        "source_project_id": source_project_id,
        "source_dataset_id": source_dataset_id,
    },
    schedule="every 24 hours",
)
transfer_config = transfer_client.create_transfer_config(
    parent=transfer_client.common_project_path(destination_project_id),
    transfer_config=transfer_config,
)
print(f"Created transfer config: {transfer_config.name}")

Created transfer config: projects/529423390811/locations/europe-west2/transferConfigs/64ff46a0-0000-2d9c-b971-089e0820ebe0


### Building the container images

The [model/](/model/) directory contains the code for custom training and serving container images [model/training/train.py](model/training/train.py).

Build the training and serving container images and push them to Artifact Registry.

In [ ]:
! make build

Building training image
Creating temporary tarball archive of 7 file(s) totalling 87.1 KiB before compression.
Uploading tarball of [.] to [gs://dt-donatas-sandbox-dev-staging/source/1694009873.552364-fb32142e7a9740909e0a330ba3cfe17c.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/dt-donatas-sandbox-dev/locations/europe-west2/builds/8186df5e-46d3-4ff7-909b-1d5fb539e3a1].
Logs are available at [ https://console.cloud.google.com/cloud-build/builds;region=europe-west2/8186df5e-46d3-4ff7-909b-1d5fb539e3a1?project=529423390811 ].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "8186df5e-46d3-4ff7-909b-1d5fb539e3a1"

FETCHSOURCE
Fetching storage object: gs://dt-donatas-sandbox-dev-staging/source/1694009873.552364-fb32142e7a9740909e0a330ba3cfe17c.tgz#1694009874031650
Copying gs://dt-donatas-sandbox-dev-staging/source/1694009873.552364-fb32142e7a9740909e0a330ba3cfe17c.tgz#1694009874031650...
/ [1 files][ 28.4 KiB/ 28.4 KiB]           

### Running Pipelines

You can run the training pipeline (for example) with:

This will execute the pipeline using the chosen template on Vertex AI, namely it will:

1. Compile the pipeline using the Kubeflow Pipelines SDK
1. Trigger the pipeline with the help of `pipelines/trigger/main.py`

**Run Training Pipeline**

In [ ]:
! make run pipeline=training build=false

**Run Prediction Pipeline** 

After successful training run you can try prediction pipeline.

In [ ]:
! make run pipeline=prediction build=false

## 4. Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

{TODO: Include infrastructure cleanup}